In [1]:
import matplotlib.pyplot as plt
import numpy as np
# import IPython.display as disp
import geopandas as gpd
from geemap import geopandas_to_ee
import pandas as pd
import requests
import logging
import ee
import xarray as xr
import numpy as np
import rasterio
from datetime import datetime
from pygeosys.util.dataframe import chunk_dataframe
import folium

In [48]:
    # -------------------------------------------------------------------------

    # GEE logging & initialization
    # -------------------------------------------------------------------------

service_account = 'aubin-allies-sandbox@earthengine-185413.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/home/lwh/Documents/aubin-allies-sandbox-earthengine-185413-d486e9739448.json')
ee.Initialize(credentials)
logger = logging.getLogger()
    # 

# Sentinel-2

In [22]:
# Constants
# -----------------------------------------------------------------------------

# Parameters for cloud masking
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

# Year of extraction
year=2020

In [23]:
# S2 collection with cloud probability
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [24]:
# Add clouds band
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [25]:
# Add clouds shadows band
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [26]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [27]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [28]:
# Functions GEE display with folium
# -----------------------------------------------------------------------------
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [10]:
def NDVI(image):
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))
def NBR2(image):
    return image.addBands(image.normalizedDifference(['B11', 'B12']).rename('NBR2'))

In [34]:
# ee.Initialize()
# logger = logging.getLogger()
# -----------------------------------------------------------------------------

# Define input parameters
# -----------------------------------------------------------------------------
root='/home/lwh/Documents/PROJECTS/Bare_soil_detection/bare_soil_detection_test/data/00_external/test_Pierre/'
input_filepath = root+'G78/G78.shp'
chunk_size = 1


# load input vector dataset
logger.info('Loading FeatureCollection')
geometry_collection = gpd.read_file(input_filepath)

if chunk_size is None:
    geometry_collections = [geometry_collection]
else:
    geometry_collections = list(chunk_dataframe(geometry_collection, chunk_size))

logger.info(f'Chunks: {len(geometry_collections)}')
filename_prefixes = []

output_folder='/home/lwh/Documents/PROJECTS/Bare_soil_detection/bare_soil_detection_test/data/04_features/'
year=2021

# loop on chunks
for chunk_id,  gc in enumerate(geometry_collections):
    logger.info(
        f'Processing chunk {chunk_id} / {len(geometry_collections)}')
    logger.info(
        f'Uploading FeatureCollection ({len(gc)} Features) on server side')
    feature_collection_ee = geopandas_to_ee(gc)

    logger.info('Creating Task')

    feature = feature_collection_ee.first()
    
    s2_sr_cld_col = get_s2_sr_cld_col(feature.geometry(), f'{year}-06-01', f'{year}-06-15')
    s2_sr_cld_col=s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).map(NDVI).map(NBR2)
    im1 = ee.Image(s2_sr_cld_col.filterDate(f'{year}-06-06', f'{year}-06-10').select('NDVI').median())
    im2 = ee.Image(s2_sr_cld_col.filterDate(f'{year}-06-06', f'{year}-06-10').select('NBR2').median())
    Im_final=im1.addBands(im2)
    output_filename = f"bare_soil_detection/S2image_{year}_06_15"
    fullName=f"bare_soil_detection_test_Pierre_S2"
    
    task=ee.batch.Export.image.toCloudStorage(**{
        'image': Im_final,
        'description': fullName,
        'bucket': 'gri_geosys',
        'fileNamePrefix':output_filename,
        'scale': 10,
        'region':feature.geometry(),
        'maxPixels': 15000000000,
        'skipEmptyTiles': True
        })
    task.start()
    


In [31]:
import time
acq_times = s2_sr_cld_col=s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).map(NDVI).map(NBR2).aggregate_array('GENERATION_TIME').getInfo()
# [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]


In [32]:
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['06/04/21', '06/09/21', '06/14/21']

In [33]:
im1 = ee.Image(col.filterDate(f'{year}-06-07', f'{year}-06-09').select('NDVI').select('NBR2').median())

NameError: name 'col' is not defined

# Landsat-8

In [35]:
def clearNDVI_L8(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
  #  CloudMask
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  #  Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  #  Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).bitwiseAnd(cloudsBitMask).eq(0)
  return image.addBands(ndvi.updateMask(mask));#.updateMask(qual)

def clearNBR2_L8(image):
  
  #  CloudMask
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  #  Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  #  Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).bitwiseAnd(cloudsBitMask).eq(0)
  image=image.updateMask(mask)
  nbr2 =  image.addBands(image.normalizedDifference(['SR_B6', 'SR_B7']).rename('NBR2'))
  ndvi =  nbr2.addBands(nbr2.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))
  return ndvi;#.updateMask(qual)

band_red = 'SR_B4'
band_nir = 'SR_B5'
band_swir1='SR_B6'
band_swir2='SR_B7'
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(f'{year}-05-15', f'{year}-06-15').filterBounds(feature_collection_ee).select(band_red, band_nir,band_swir1,band_swir2, 'QA_PIXEL', 'QA_RADSAT')
ndvi2 = col.map(clearNBR2_L8)
output_filename = f"bare_soil_detection/L8image_{year}_06_15"
fullName=f"bare_soil_detection_test_Pierre_L8_2021"



In [36]:
import time
acq_times = col.aggregate_array('DATE_PRODUCT_GENERATED').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]


['05/29/21', '06/15/21', '06/08/21', '06/22/21', '06/22/21']

In [37]:
im1 = ee.Image(ndvi2.filterDate(f'{year}-06-07', f'{year}-06-10').select('NDVI').median())
im2 = ee.Image(ndvi2.filterDate(f'{year}-06-07', f'{year}-06-10').select('NBR2').median())
Im_final=im1.addBands(im2)

In [38]:
Im_final.bandNames().getInfo()

['NDVI', 'NBR2']

In [39]:
task=ee.batch.Export.image.toCloudStorage(**{
    'image': Im_final,
    'description': fullName,
    'bucket': 'gri_geosys',
    'fileNamePrefix':output_filename,
    'scale': 10,
    'region':feature.geometry(),
    'maxPixels': 15000000000,
    'skipEmptyTiles': True
    })
task.start()

# Sentinel 1 

In [40]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline
#Pandas modules to interact with spatial data
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
from geemap import geopandas_to_ee

from pygeosys.util.dataframe import chunk_dataframe
#import numpy
import numpy as np
from functools import reduce

In [41]:
 # Import the Folium library.
import folium
from change_detection_tools import add_ee_layer
# Define a method for displaying Earth Engine image tiles to folium map.
# def add_ee_layer(self, ee_image_object, vis_params, name):
#   map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
#   folium.raster_layers.TileLayer(
#     tiles = map_id_dict['tile_fetcher'].url_format,
#     attr = 'Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     name = name,
#     overlay = True,
#     control = True
#   ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.

In [45]:
year=2021
date_start = f'{year}-05-10' # fall tillage : 08-01 to 11-01 ; spring tillage : 02-01 to 05-01
date_end = f'{year}-06-12'

chunk_size=3
if chunk_size is None:
    geometry_collections = [geometry_collection]
else:
    geometry_collections = list(chunk_dataframe(geometry_collection, chunk_size))

print(f'Chunks: {len(geometry_collections)}')
filename_prefixes = []
for chunk_id,  gc in enumerate(geometry_collections):
    print(
            f'Processing chunk {chunk_id} / {len(geometry_collections)}')
    print(
            f'Uploading FeatureCollection ({len(gc)} Features) on server side')
    table = geopandas_to_ee(gc)


aoi = table.geometry()
aoi_sub=table.first().geometry()

Chunks: 1
Processing chunk 0 / 1
Uploading FeatureCollection (1 Features) on server side


In [49]:
from change_detection_tools import get_available_orbits
# Test fpr available orbit
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date(date_start),ee.Date(date_end))
           .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')))

# im_coll.aggregate_array('relativeOrbitNumber_start').getInfo()
print (get_available_orbits(im_coll))

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.

In [50]:
dataset=ee.ImageCollection('COPERNICUS/S1_GRD')
im_coll = dataset.filterBounds(
    aoi_sub).filterDate(ee.Date(date_start),ee.Date(date_end)
    ).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
    ).sort('system:time_start')

In [51]:
im_list = im_coll.toList(im_coll.size())
im_list.get(0)

In [52]:
im_coll.aggregate_array('relativeOrbitNumber_start').getInfo()


[34, 136, 34, 136, 34, 136]

In [53]:
import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['05/10/21', '05/17/21', '05/22/21', '05/29/21', '06/03/21', '06/10/21']

In [54]:
def RVI(image):
    return image.addBands(image.expression('4*(10**(VH/10))/((10**(VH/10))+(10**(VV/10)))', {
                'VH': image.select('VH'),
                'VV': image.select('VV'), }).rename('RVI'))

In [55]:
im_coll=im_coll.map(RVI)
im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(4)).select('RVI').clip(aoi_sub)
# im2 = ee.Image(im_list.get(6)).select('VV').clip(aoi_sub)

In [56]:
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=20)
mp.add_ee_layer(im1,
                {'min': 0, 'max': 10, 'palette': ['black', 'white']}, 'Ratio')
mp.add_child(folium.LayerControl())

display(mp)
url = im1.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)

In [57]:
output_filename = f"bare_soil_detection/S1image_{year}_06_15"
fullName=f"bare_soil_detection_test_Pierre_S1_2021"
task=ee.batch.Export.image.toCloudStorage(**{
    'image': im1,
    'description': fullName,
    'bucket': 'gri_geosys',
    'fileNamePrefix':output_filename,
    'scale': 10,
    'region':feature.geometry(),
    'maxPixels': 15000000000,
    'skipEmptyTiles': True
    })
task.start()